# Pre-processing

In [1]:
import sys, os, glob, shutil
sys.path.append('../')

from MRIsegm.utils import get_slice, get_slice_info, get_rois, explore_slice, explore_roi, make_mask, mask_slice
from MRIsegm.processing import denoise_slice

from PIL import Image
from tqdm.notebook import trange



## Loading Data

In [2]:
src = '/Users/giuseppefilitto/Pazienti_anonym_sorted'

patients = os.listdir(src)
if '.DS_Store' in patients:
    patients.remove('.DS_Store')

bad_patients = [ 'BO9', 'BO17', 'BO28', 'BO36', 'BO37', 'BO39', 'BO40', 'BO54', 'BO72', 'BO77', 'BO86']

good_patients = list(set(patients) - set(bad_patients))

# ! Removing for now 
good_patients.remove('BO38')
print("Number of good patients:",len(good_patients))

Number of good patients: 36


## Saving Train and Test images

### Training

In [3]:
progress_bar = trange(len(good_patients), desc="Progress")
for patient in good_patients:

    #! slices
    folder = 'T2'
    slice_path = os.path.join(src, patient, folder)

    if not os.path.isdir(slice_path):
        slice_path =  slice_path + "AX"

        if not os.path.isdir(slice_path):
            slice_path = os.path.join(src, patient, 'T25mm')

            if not os.path.isdir(slice_path):
                slice_path = os.path.join(src, patient, 't2DEF')


    slice_raw = get_slice(dir_path=slice_path)  
    slice = denoise_slice(slice_raw, alpha = 7)

    roi_folder = 'T2ROI'
    roi_path = os.path.join(src, patient, roi_folder)

    #! ROIS
    roi = get_rois(roi_path=roi_path)
    

    #! slits of masks

    slice_of_masks = mask_slice(slice=slice, rois=roi)
    

    positions = [roi[j].get('position') - 1 for j in range(len(roi))]

    
    for i in set(positions):

        img = Image.fromarray(slice[i, : ,:])
        filename = patient + '_layer_' + str(i) + '.png'
        dst = '../data/training/img/img'
        output =  os.path.join(dst, filename)

        if os.path.isfile(output):
            os.remove(output)
        
        img.save(output)

        mask = Image.fromarray(slice_of_masks[i, : ,:])
        filename = patient + '_mask_' + str(i) + '.png'
        dst = '../data/training/mask/img'
        output =  os.path.join(dst, filename)
        if os.path.isfile(output):
            os.remove(output)

        mask.save(output)
 
    progress_bar.update(1)    

progress_bar.close()    

Progress:   0%|          | 0/36 [00:00<?, ?it/s]

### Test

In [4]:
train_dir = '../data/training/img/img'
train_files = glob.glob(train_dir + '/*.png')

mask_dir = '../data/training/mask/img'
mask_files = glob.glob(mask_dir + '/*.png')



# ! test
test_dir = '../data/test/img/img'
mask_test_dir = '../data/test/mask/img'

n_test = int(len(train_files) * 0.2)
print("Training images:", len(train_files) - n_test)
print("Test images:", n_test)


Training images: 376
Test images: 93


In [5]:
for i in range(0,n_test + 1):
    
    dir, file = os.path.split(train_files[i])
    test_img = os.path.join(test_dir, file)

    img = train_files[i]
    if os.path.isfile(test_img):
        os.remove(test_img)

    shutil.move(img, test_img)

    mask_dir = '../data/training/mask/img'
    mask_file = file.replace('_layer_','_mask_')
    
    mask = os.path.join(mask_dir, mask_file)
    mask_test = os.path.join(mask_test_dir, file)
    if os.path.isfile(mask_test):
        os.remove(mask_test)

    shutil.move(mask, mask_test)